In [1]:
from arrest.handler import ResourceHandler
from arrest import Service, Resource
import re

In [10]:
svc = Service(
    name="xyz",
    url="http://example.com",
    resources=[
        Resource(
            route="/user",
            handlers=[
                ("GET", ""),
                ("GET", "/"),
            ],
        )
    ],
)

In [12]:
svc.user.get_matching_handler(method="GET", path="")

self.path_regex.fullmatch(path)=<re.Match object; span=(0, 0), match=''>


(ResourceHandler(method=GET, route='', request=None, response=None, callback=None, path_format='', path_regex=re.compile('^$'), param_types={}),
 'http://example.com/user')

In [1]:
from pydantic import BaseModel, ValidationError

class A(BaseModel):
    a: str
    b: int


try:
    A(b=2)
except ValidationError as ex:
    print(ex.errors())

[{'type': 'missing', 'loc': ('a',), 'msg': 'Field required', 'input': {'b': 2}, 'url': 'https://errors.pydantic.dev/2.5/v/missing'}]
